In [3]:
from datetime import datetime
import lxml
from lxml import html
import requests
import numpy as np
import pandas as pd

In [4]:
symbol = 'AAPL'

In [5]:
url_bs = 'https://finance.yahoo.com/quote/' + symbol + '/balance-sheet?p=' + symbol
url_is = 'https://finance.yahoo.com/quote/' + symbol + '/financials?p=' + symbol
url_cf = 'https://finance.yahoo.com/quote/' + symbol + '/cash-flow?p='+ symbol

In [6]:
headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
    'Cache-Control': 'max-age=0',
    'Pragma': 'no-cache',
    'Referrer': 'https://google.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36'
}

In [7]:
def get_table(url):
    # Fetch the page that we're going to parse, using the request headers defined above
    page = requests.get(url, headers)

    # Parse the page with LXML, so that we can start doing some XPATH queries
    # to extract the data that we want
    tree = html.fromstring(page.content)

    # Smoke test that we fetched the page by fetching and displaying the H1 element
    tree.xpath("//h1/text()")
    table_rows = tree.xpath("//div[contains(@class, 'D(tbr)')]")

    # Ensure that some table rows are found; if none are found, then it's possible
    # that Yahoo Finance has changed their page layout, or have detected
    # that you're scraping the page.
    assert len(table_rows) > 0

    parsed_rows = []

    for table_row in table_rows:
        parsed_row = []
        el = table_row.xpath("./div")

        none_count = 0

        for rs in el:
            try:
                (text,) = rs.xpath('.//span/text()[1]')
                parsed_row.append(text)
            except ValueError:
                parsed_row.append(np.NaN)
                none_count += 1

        if (none_count < 4):
            parsed_rows.append(parsed_row)

    df = pd.DataFrame(parsed_rows)
    df_org = df
    
    df = pd.DataFrame(parsed_rows)
    df = df.set_index(0) # Set the index to the first column: 'Period Ending'.
    df = df.transpose() # Transpose the DataFrame, so that our header contains the account names

    # Rename the "Breakdown" column to "Date"
    cols = list(df.columns)
    cols[0] = 'Date'
    df = df.set_axis(cols, axis='columns', inplace=False)
    df_rot = df
    return df_org, df_rot

In [8]:
# get Balance Sheet 
BS_orginal, BS_transpose = get_table(url_bs)

# get Income Statement 
IS_orginal, IS_transpose = get_table(url_is)

# get Cash Flow
CF_orginal, CF_transpose = get_table(url_cf)

In [9]:
BS_orginal

,0,1,2,3,4
0,Breakdown,9/30/2020,9/30/2019,9/30/2018,9/30/2017
1,Total Assets,"323,888,000","338,516,000","365,725,000","375,319,000"
2,Total Liabilities Net Minority Interest,"258,549,000","248,028,000","258,578,000","241,272,000"
3,Total Equity Gross Minority Interest,"65,339,000","90,488,000","107,147,000","134,047,000"
4,Total Capitalization,"164,006,000","182,295,000","200,882,000","231,254,000"
5,Common Stock Equity,"65,339,000","90,488,000","107,147,000","134,047,000"
6,Net Tangible Assets,"65,339,000","90,488,000","107,147,000","126,032,000"
7,Working Capital,"38,321,000","57,101,000","14,473,000","27,831,000"
8,Invested Capital,"177,775,000","198,535,000","221,630,000","249,727,000"
9,Tangible Book Value,"65,339,000","90,488,000","107,147,000","126,032,000"


In [10]:
BS_transpose

,Date,Total Assets,Total Liabilities Net Minority Interest,Total Equity Gross Minority Interest,Total Capitalization,Common Stock Equity,Net Tangible Assets,Working Capital,Invested Capital,Tangible Book Value,Total Debt,Net Debt,Share Issued,Ordinary Shares Number
1,9/30/2020,"323,888,000","258,549,000","65,339,000","164,006,000","65,339,000","65,339,000","38,321,000","177,775,000","65,339,000","112,436,000","74,420,000","16,976,763","16,976,763"
2,9/30/2019,"338,516,000","248,028,000","90,488,000","182,295,000","90,488,000","90,488,000","57,101,000","198,535,000","90,488,000","108,047,000","59,203,000","17,772,944","17,772,944"
3,9/30/2018,"365,725,000","258,578,000","107,147,000","200,882,000","107,147,000","107,147,000","14,473,000","221,630,000","107,147,000","114,483,000","88,570,000","19,019,944","19,019,944"
4,9/30/2017,"375,319,000","241,272,000","134,047,000","231,254,000","134,047,000","126,032,000","27,831,000","249,727,000","126,032,000","115,680,000","95,391,000","20,504,804","20,504,804"


In [11]:
# Balance Sheet Analysis
BS_analysis = pd.DataFrame(BS_transpose['Date']) # copy columns of dataframe
BS_analysis

,Date
1,9/30/2020
2,9/30/2019
3,9/30/2018
4,9/30/2017


# Run later 

In [ ]:
# working capital = current assets - current liabilities

current_assets = BS_transpose['Total Current Assets'].str.replace(',', '').astype(int)
current_liabilities = BS_transpose['Total Current Liabilities'].str.replace(',', '').astype(int)
working_capital = current_assets - current_liabilities
working_capital
BS_analysis['Working Capital'] = working_capital # copy columns of dataframe
BS_analysis

In [ ]:
# Working Capital Per Dollar of Sales = Working Capital ÷ Total Sales
# Total Sales is from Income Statement

current_assets = BS_transpose['Total Current Assets'].str.replace(',', '').astype(int)
current_liabilities = BS_transpose['Total Current Liabilities'].str.replace(',', '').astype(int)
working_capital = current_assets - current_liabilities
working_capital
total_sales = IS_transpose['Total Revenue'].str.replace(',', '').astype(int)
working_capital_per_dollar_of_sales = working_capital / total_sales
BS_analysis['Working Capital per Dollar of Sales'] = working_capital # copy columns of dataframe
BS_analysis

In [ ]:
# Current Ratio = Current Assets ÷ Current Liabilities

current_ratio = current_assets / current_liabilities
BS_analysis['Current Ratio'] = current_ratio
BS_analysis

In [ ]:
# Quick / Acid Test / Current Ratio = Current Assets minus inventory (called "Quick Assets) ÷ Current Liabilities

inventory = BS_transpose['Inventory'].str.replace(',', '').astype(int)
quick_current_ratio = (current_assets - inventory) / current_liabilities
BS_analysis['Quick Current Ratio'] = quick_current_ratio
BS_analysis

In [ ]:
# debt-to-equity ratio = total liabilities ÷ shareholders' equity

total_liabilities = BS_transpose['Total Liabilities'].str.replace(',', '').astype(int)
shareholders_equity = BS_transpose['Total stockholders\' equity'].str.replace(',', '').astype(int)
debt2equity_ratio = total_liabilities / shareholders_equity
BS_analysis['Debt to Equity Ratio'] = debt2equity_ratio
BS_analysis

In [ ]:
# Receivable Turnover = Net Credit Sales ÷ Average Net Receivables for the Period
# Net Credit Sales is from Income Statement

net_credit_sales = IS_transpose['Net Income'].str.replace(',', '').astype(int)
average_net_receivables_for_the_period = BS_transpose['Net Receivables'].str.replace(',', '').astype(int)
receivable_turnover = net_credit_sales / average_net_receivables_for_the_period
BS_analysis['Receivable Turnover'] = receivable_turnover
BS_analysis

In [ ]:
# Average Age of Receivables = Number of days in period ÷ Receivable Turnover

number_of_days_in_period = 365
average_age_of_receivables = number_of_days_in_period / receivable_turnover
BS_analysis['Average Age of Receivables'] = average_age_of_receivables
BS_analysis

In [ ]:
# Inventory Turnover = Cost of Goods Sold ÷ Average Inventory for the Period
# Cost of Goods Sold is from Income Statement

cost_of_goods_sold = IS_transpose['Cost of Revenue'].str.replace(',', '').astype(int)
average_inventory_for_the_period = inventory
inventory_turnover = cost_of_goods_sold / average_inventory_for_the_period
BS_analysis['Inventory Turnover'] = inventory_turnover
BS_analysis

In [ ]:
# Number of Days for Inventory to Turn = Number of days in Period ÷ Inventory Turnover

number_of_days_for_inventory_to_turn = number_of_days_in_period / inventory_turnover
BS_analysis['Number of Days for Inventory to Turn'] = number_of_days_for_inventory_to_turn
BS_analysis

In [12]:
# Income Statement original table
IS_orginal

,0,1,2,3,4,5
0,Breakdown,ttm,9/30/2020,9/30/2019,9/30/2018,9/30/2017
1,Total Revenue,"273,857,000","274,515,000","260,174,000","265,595,000","229,234,000"
2,Cost of Revenue,"169,277,000","169,559,000","161,782,000","163,756,000","141,048,000"
3,Gross Profit,"104,580,000","104,956,000","98,392,000","101,839,000","88,186,000"
4,Operating Expense,"37,442,000","38,668,000","34,462,000","30,941,000","26,842,000"
5,Operating Income,"67,138,000","66,288,000","63,930,000","70,898,000","61,344,000"
6,Net Non Operating Interest Income Expense,"1,052,000","890,000","1,385,000","2,446,000","2,878,000"
7,Other Income Expense,"127,000","-87,000","422,000","-441,000","-133,000"
8,Pretax Income,"68,317,000","67,091,000","65,737,000","72,903,000","64,089,000"
9,Tax Provision,"9,893,000","9,680,000","10,481,000","13,372,000","15,738,000"


In [13]:
# Income Statement transpose table
IS_transpose

,Date,Total Revenue,Cost of Revenue,Gross Profit,Operating Expense,Operating Income,Net Non Operating Interest Income Expense,Other Income Expense,Pretax Income,Tax Provision,...,Interest Income,Interest Expense,Net Interest Income,EBIT,Reconciled Cost of Revenue,Reconciled Depreciation,Net Income from Continuing Operation Net Minority Interest,Normalized EBITDA,Tax Rate for Calcs,Tax Effect of Unusual Items
1,ttm,"273,857,000","169,277,000","104,580,000","37,442,000","67,138,000","1,052,000","127,000","68,317,000","9,893,000",...,"4,101,000","3,049,000","1,052,000","71,366,000","169,277,000","11,533,000","58,424,000","82,899,000",0,0
2,9/30/2020,"274,515,000","169,559,000","104,956,000","38,668,000","66,288,000","890,000","-87,000","67,091,000","9,680,000",...,"3,763,000","2,873,000","890,000","69,964,000","169,559,000","11,056,000","57,411,000","81,020,000",0,0
3,9/30/2019,"260,174,000","161,782,000","98,392,000","34,462,000","63,930,000","1,385,000","422,000","65,737,000","10,481,000",...,"4,961,000","3,576,000","1,385,000","69,313,000","161,782,000","12,547,000","55,256,000","81,860,000",0,0
4,9/30/2018,"265,595,000","163,756,000","101,839,000","30,941,000","70,898,000","2,446,000","-441,000","72,903,000","13,372,000",...,"5,686,000","3,240,000","2,446,000","76,143,000","163,756,000","10,903,000","59,531,000","87,046,000",0,0
5,9/30/2017,"229,234,000","141,048,000","88,186,000","26,842,000","61,344,000","2,878,000","-133,000","64,089,000","15,738,000",...,"5,201,000","2,323,000","2,878,000","66,412,000","141,048,000","10,157,000","48,351,000","76,569,000",0,0


In [14]:
#Income statement Analysis
IS_analysis = pd.DataFrame(IS_transpose['Date']) # copy columns of dataframe
IS_analysis

,Date
1,ttm
2,9/30/2020
3,9/30/2019
4,9/30/2018
5,9/30/2017


In [ ]:
# Gross Profit Margin = (Revenue - Cost of Goods sold) ÷ Revenue

revenue = IS_transpose['Total Revenue'].str.replace(',', '').astype(int)
cost_of_goods_sold = IS_transpose['Cost of Revenue'].str.replace(',', '').astype(int)
gross_profit_margin = (revenue - cost_of_goods_sold) / revenue
IS_analysis['Gross Profit Margin'] = gross_profit_margin * 100
IS_analysis

In [ ]:
# Research and development (R&D) to sales = R&D expense ÷ Revenue

RD_expense = IS_transpose['Research Development'].str.replace(',', '').astype(int)
RD_to_sales = RD_expense / revenue
IS_analysis['RD to sales'] = RD_to_sales
IS_analysis

In [ ]:
# Operating Profit Margin = Operating Income ÷ revenue

operating_income = IS_transpose['Operating Income or Loss'].str.replace(',', '').astype(int)
operating_profit_margin = operating_income / revenue
IS_analysis['Operating Profit Margin'] = RD_to_sales
IS_analysis

In [ ]:
# Interest coverage ratio = earnings before interest and taxes (EBIT) ÷ interest expense

interest_expense = IS_transpose['Interest Expense'].str.replace(',', '').astype(int)
earnings_before_interest_and_taxes = IS_transpose['Income Before Tax'].str.replace(',', '').astype(int)
interest_coverage_ratio = earnings_before_interest_and_taxes / interest_expense
IS_analysis['Interest Coverage Ratio'] = interest_coverage_ratio
IS_analysis

In [ ]:
# Net profit margin = net income (after taxes) ÷ revenue

net_income = IS_transpose['Net Income'].str.replace(',', '').astype(int)
net_profit_margin = net_income / revenue
IS_analysis['Net Profit Ratio'] = net_profit_margin
IS_analysis

In [ ]:
# Return on equity (ROE) = net profit ÷ average shareholder equity for the period

net_profit = IS_transpose['Net Income available to common shareholders'].str.replace(',', '').astype(int)
average_shareholde_equity_for_the_period = BS_transpose['Total stockholders\' equity'].str.replace(',', '').astype(int)
return_on_equity = net_profit / average_shareholde_equity_for_the_period
IS_analysis['Return on Equity'] = return_on_equity
IS_analysis

In [ ]:
# Asset Turnover = Revenue ÷ Average Assets for the period

average_assets_for_the_period = BS_transpose['Total Assets'].str.replace(',', '').astype(int)
assets_turnover = revenue / average_assets_for_the_period
IS_analysis['Asset Turnover'] = assets_turnover
IS_analysis

In [ ]:
# Return on assets = net income ÷ total average assets for the period
# Return on assets = net profit margin x assets turnover

return_on_assets = net_profit_margin / assets_turnover
IS_analysis['Return on Assets'] = return_on_assets
IS_analysis

In [15]:
# Cash Flow original table
CF_orginal

,0,1,2,3,4,5
0,Breakdown,ttm,9/30/2020,9/30/2019,9/30/2018,9/30/2017
1,Operating Cash Flow,"80,008,000","80,674,000","69,391,000","77,434,000","63,598,000"
2,Investing Cash Flow,"-10,618,000","-4,289,000","45,896,000","16,066,000","-46,446,000"
3,Financing Cash Flow,"-86,502,000","-86,820,000","-90,976,000","-87,876,000","-17,347,000"
4,End Cash Position,"35,039,000","39,789,000","50,224,000","25,913,000","20,289,000"
5,Income Tax Paid Supplemental Data,"11,878,000","9,501,000","15,263,000","10,417,000","11,591,000"
6,Interest Paid Supplemental Data,"3,135,000","3,002,000","3,423,000","3,022,000","2,092,000"
7,Capital Expenditure,"-8,302,000","-7,309,000","-10,495,000","-13,313,000","-12,795,000"
8,Issuance of Capital Stock,"820,000","880,000","781,000","669,000","555,000"
9,Issuance of Debt,"22,794,000","16,091,000","6,963,000","6,969,000","28,662,000"


In [16]:
# Cash Flow transpose table
CF_transpose

,Date,Operating Cash Flow,Investing Cash Flow,Financing Cash Flow,End Cash Position,Income Tax Paid Supplemental Data,Interest Paid Supplemental Data,Capital Expenditure,Issuance of Capital Stock,Issuance of Debt,Repayment of Debt,Repurchase of Capital Stock,Free Cash Flow
1,ttm,"80,008,000","-10,618,000","-86,502,000","35,039,000","11,878,000","3,135,000","-8,302,000","820,000","22,794,000","-13,908,000","-72,615,000","71,706,000"
2,9/30/2020,"80,674,000","-4,289,000","-86,820,000","39,789,000","9,501,000","3,002,000","-7,309,000","880,000","16,091,000","-12,629,000","-72,358,000","73,365,000"
3,9/30/2019,"69,391,000","45,896,000","-90,976,000","50,224,000","15,263,000","3,423,000","-10,495,000","781,000","6,963,000","-8,805,000","-66,897,000","58,896,000"
4,9/30/2018,"77,434,000","16,066,000","-87,876,000","25,913,000","10,417,000","3,022,000","-13,313,000","669,000","6,969,000","-6,500,000","-72,738,000","64,121,000"
5,9/30/2017,"63,598,000","-46,446,000","-17,347,000","20,289,000","11,591,000","2,092,000","-12,795,000","555,000","28,662,000","-3,500,000","-32,900,000","50,803,000"


In [17]:
#Cash Flow Analysis
CF_analysis = pd.DataFrame(CF_transpose['Date']) # copy columns of dataframe
CF_analysis

,Date
1,ttm
2,9/30/2020
3,9/30/2019
4,9/30/2018
5,9/30/2017


In [ ]:
# Current Liability Coverage Ratio = Net Cash from Operating Activities ÷ Average Current Liabilities

net_cash_provided_from_operating_activites = CF_transpose['Net cash provided by operating activites'].str.replace(',', '').astype(int)
average_current_liabilities = BS_transpose['Total Current Liabilities'].str.replace(',', '').astype(int)
current_liability_coverage_ratio = net_cash_provided_from_operating_activites / average_current_liabilities
CF_analysis['Current Liability Coverage Ratio'] = current_liability_coverage_ratio
CF_analysis

In [ ]:
# Price to Cash Flow Ratio = Share Price ÷ Operating Cash Flow per Share

share_price = 1428.96
operating_cash_flow = CF_transpose['Operating Cash Flow'].str.replace(',', '').astype(int)
common_stock = BS_transpose['Common Stock'].str.replace(',', '').astype(int)
operating_cash_flow_per_share = operating_cash_flow / common_stock
price_to_cash_flow_ratio = share_price /  operating_cash_flow_per_share
CF_analysis['Price to Cash Flow Ratio'] = price_to_cash_flow_ratio
CF_analysis

In [ ]:
# Cash Flow Margin Ratio = Cash Flow from Operating Cash Flows ÷ Net Sales

cash_flow_margin_ratio = operating_cash_flow / net_sales
CF_analysis['Cash Flow Margin Ratio'] = cash_flow_margin_ratio
CF_analysis

In [ ]:
# Cash Flow Coverage Ratio = Cash Flow from Operations ÷ Total Debt

cash_flow_coverage_ratio = cash_flow_from_operations / total_debt
CF_analysis['Cash Flow Margin Ratio'] = cash_flow_coverage_ratio
CF_analysis